In [4]:
Dropout_model = 0.3856
FVC_weight = 0.2
Confidence_weight = 0
#!pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index
#!pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index
import os
#import cv2
import pydicom
import pandas as pd
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt 
import random
from tqdm.notebook import tqdm 
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from tensorflow_addons.optimizers import RectifiedAdam
from tensorflow.keras import Model
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.optimizers import Nadam
import seaborn as sns
from PIL import Image

def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    
seed_everything(45)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

tr = pd.read_excel(open('../input/dryierbindatafornurserycorn/Drying formula.xlsx', 'rb'),
              sheet_name='Sheet2') 

chunk =  tr.loc[tr.Datetype == "Test"] 
tr["ID"] = range(1, 1 + len(tr))
chunk["a"] = "sub"
chunk["b"] = range(1, 1 + len(chunk))
chunk["ID"] = chunk["a"] + chunk["b"].map(str)

del chunk["a"]
del chunk["b"]
print(chunk)

#print(tr)
#len(tr)
#tr=tr.drop_duplicates(keep=False, inplace=True, subset=['ID','totalDays'])
sub = pd.read_excel(open('../input/submission-dryier/Drying formula -sub.xlsx', 'rb'),
              sheet_name='Submission') 
print("add infos")
sub['ID'] = sub['ID_day'].apply(lambda x:x.split('_')[0])
sub['totalDays'] = sub['ID_day'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['ID','totalDays','Confidence','ID_day']]
#del sub['ID']
#sub["ID"] = range(101, 101 + len(sub))

tr = tr.dropna(how='any', subset=['moistureOffPercent','moistureOnPercent'])

print(sub)

ModuleNotFoundError: No module named 'tensorflow_addons'

In [ ]:
sub.head()

In [ ]:
BATCH_SIZE=128
#df.year.astype(int)
#tr.drop_duplicates(keep=False, inplace=True, subset=['ID','totalDays'])

sub = sub.merge(chunk.drop('totalDays', axis=1), on="ID")
tr['WHERE'] = 'train'
chunk['WHERE'] = 'val'
sub['WHERE'] = 'test'

In [ ]:
#tr.head()
sub

In [ ]:
data = tr.append([chunk,sub])
print(tr.shape, chunk.shape, sub.shape, data.shape)
print(tr.ID.nunique(), chunk.ID.nunique(), sub.ID.nunique(), 
      data.ID.nunique())

data['min_day'] = data['totalDays']
data.loc[data.WHERE=='test','min_day'] = np.nan
data['min_day'] = data.groupby('ID')['min_day'].transform('min')
base = data.loc[data.totalDays == data.min_day]
base = base[['ID','moistureOffPercent']].copy()
base.columns = ['ID','min_moistureOffPercent']
base['nb'] = 1
base['nb'] = base.groupby('ID')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)
data = data.merge(base, on='ID', how='left')
data['base_week'] = data['totalDays'] - data['min_day']
del base

data.head()
data.shape

In [ ]:
COLS = ['Year'] #,'Age'
FE = []
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)

In [ ]:
data['Moist'] = (data['moistureOnPercent'] - data['moistureOnPercent'].min() ) / ( data['moistureOnPercent'].max() - data['moistureOnPercent'].min() )
#data['BASE'] = (data['min_FVC'] - data['min_FVC'].min() ) / ( data['min_FVC'].max() - data['min_FVC'].min() )
#data['week'] = (data['base_week'] - data['base_week'].min() ) / ( data['base_week'].max() - data['base_week'].min() )
#data['percent'] = (data['Percent'] - data['Percent'].min() ) / ( data['Percent'].max() - data['Percent'].min() )#
#data['FVC_Percent'] = data['FVC'] / data['Percent']
# FE += ['age','percent','week','BASE', 'FVC_Percent']
FE += ['Moist','moistureOnPercent']
tr = data.loc[data.WHERE=='train']
chunk = data.loc[data.WHERE=='val']
sub = data.loc[data.WHERE=='test']

#del data

In [ ]:
#for col in tr.columns: 
#    print(col) 
    
tr=tr.drop("dateOn",axis=1)
tr=tr.drop("dateOff",axis=1)
#tr=tr.drop("ID_day",axis=1)
tr=tr.drop("moistureMidPercent",axis=1)
tr=tr.drop("FLD",axis=1)
tr=tr.drop("tempF",axis=1)
tr=tr.drop("Datetype",axis=1)


#for col in chunk.columns: 
#    print(col) 
    
chunk=chunk.drop("dateOn",axis=1)
chunk=chunk.drop("dateOff",axis=1)
#chunk=chunk.drop("ID_day",axis=1)
chunk=chunk.drop("moistureMidPercent",axis=1)
chunk=chunk.drop("FLD",axis=1)
chunk=chunk.drop("tempF",axis=1)
chunk=chunk.drop("Datetype",axis=1)

#for col in sub.columns: 
#    print(col) 
    
sub=sub.drop("dateOn",axis=1)
sub=sub.drop("dateOff",axis=1)
#sub=sub.drop("ID_day",axis=1)
sub=sub.drop("moistureMidPercent",axis=1)
sub=sub.drop("FLD",axis=1)
sub=sub.drop("tempF",axis=1)
sub=sub.drop("Datetype",axis=1)


In [ ]:

C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return K.mean(metric)

def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return K.mean(v)

def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss


def make_model(nh,D,ML):
    #ML=ML
    z = L.Input((nh,), name="Patient")
    x = L.Dense(D, activation="relu", name="d1")(z)
    #x = L.Dense(D, activation="relu", name="d2")(x)
    p1 = L.Dense(3, activation="linear", name="p1")(x)
    p2 = L.Dense(3, activation="relu", name="p2")(x)
    preds = L.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                     name="preds")([p1, p2])
    
    model = M.Model(z, preds, name="CNN")
    model.compile(loss=mloss(ML), optimizer="sgd", 
                  metrics=["poisson"])
    return model

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=100,monitor="val_loss"),
    #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
     #                         patience=50, min_lr=0.001
#),
    #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]
print(FE)


In [ ]:
y = tr['moistureOffPercent'].values  # train target
z = tr[FE].values  # fetures (1535, 9)
print(z.shape)
ze = sub[FE].values  # fetures of submission (730, 9) e: estimate
print(ze.shape) 
nh = z.shape[1] 
print(nh)  # feature numbers (9,)

#with strategy.scope():    
net = make_model(nh,D=50,ML=.65)
print(net.summary())
print(net.count_params())
NFOLD = 6 # originally 5
kf = KFold(n_splits=NFOLD)

In [ ]:
del(pe)
del(pred)
pe = np.zeros((ze.shape[0], 3))  #estimate of prediction
pred = np.zeros((z.shape[0], 3))  # prediction of truth ground
cnt = 0
EPOCHS = 1000
D=100
ML=.8
BATCH_SIZE=128
y = y.astype(np.float32)
for tr_idx, val_idx in kf.split(z):
    cnt += 1
    print(f"FOLD {cnt}")
    net = make_model(nh,D,ML)
    net.fit(z[tr_idx], y[tr_idx], batch_size=BATCH_SIZE, epochs=EPOCHS, 
            validation_data=(z[val_idx], y[val_idx]), verbose=0) #
    print("train", net.evaluate(z[tr_idx], y[tr_idx], verbose=0, batch_size=BATCH_SIZE))
    print("val", net.evaluate(z[val_idx], y[val_idx], verbose=0, batch_size=BATCH_SIZE))
    print("predict val...")
    pred[val_idx] = net.predict(z[val_idx], batch_size=BATCH_SIZE, verbose=0)
    print("predict test...")
    pe += net.predict(ze, batch_size=BATCH_SIZE, verbose=0) / NFOLD
    
np.corrcoef(y, pred[:, 1])



In [ ]:
import matplotlib.pyplot as plt


plt.plot(y, pred[:, 1],"o")
plt.show()

In [ ]:
sigma_opt = mean_absolute_error(y, pred[:, 1])
unc = pred[:,2] - pred[:, 0]
sigma_mean = np.mean(unc)
print(sigma_opt, sigma_mean)
idxs = np.random.randint(0, y.shape[0], 100)
plt.plot(y[idxs], label="ground truth")
plt.plot(pred[idxs, 0], label="q25")
plt.plot(pred[idxs, 1], label="q50")
plt.plot(pred[idxs, 2], label="q75")
plt.legend(loc="best")
plt.show()
print(unc.min(), unc.mean(), unc.max(), (unc>=0).mean())
plt.hist(unc)
plt.title("uncertainty in prediction")
plt.show()
#pred

In [ ]:
#laplace_log_likelihood(y_true=tr['moistureOffPercent'], y_pred=tr['moistureOffPercent'], confidence=0)

In [ ]:
sub['moistureOffPercent1'] = 1.*pe[:, 1]
sub['Confidence1'] = pe[:, 2] - pe[:, 0]
subm = sub[['ID_day','moistureOffPercent','Confidence','moistureOffPercent1','Confidence1']].copy()
subm.loc[~subm.moistureOffPercent1.isnull()].head(10)
subm.loc[~subm.moistureOffPercent1.isnull(),'moistureOffPercent'] = subm.loc[~subm.moistureOffPercent1.isnull(),'moistureOffPercent1']
if sigma_mean<70:
    subm['Confidence'] = sigma_opt
else:
    subm.loc[~subm.moistureOffPercent1.isnull(),'Confidence'] = subm.loc[~subm.moistureOffPercent1.isnull(),'Confidence1']
subm.head()


In [ ]:
# PREDICTION
subm.describe().T
tr = pd.read_excel(open('../input/dryierbindatafornurserycorn/Drying formula.xlsx', 'rb'),
              sheet_name='Sheet2') 

otest =  tr.loc[tr.Datetype == "Test"] 
#tr["ID"] = range(1, 1 + len(tr))
otest["a"] = "sub"
otest["b"] = range(1, 1 + len(otest))
otest["ID"] = otest["a"] + otest["b"].map(str)

for i in range(len(otest)):
    subm.loc[subm['ID_day']==otest.ID[i]+'_'+str(otest.b[i]), 'moistureOffPercent'] = otest.moistureOffPercent[i]
    subm.loc[subm['ID_day']==otest.ID[i]+'_'+str(otest.b[i]), 'Confidence'] = 0.1
subm[["ID_day","moistureOffPercent","Confidence"]].to_csv("submission_regression.csv", index=False)
reg_sub = subm[["ID_day","moistureOffPercent","Confidence"]].copy()
#df1 = img_sub.sort_values(by=['ID_day'], ascending=True).reset_index(drop=True)
df2 = reg_sub.sort_values(by=['ID_day'], ascending=True).reset_index(drop=True)
#df = df1[['ID_day']].copy()
#df['FVC'] = FVC_weight*df1['moistureOffPercent'] + (1-FVC_weight)*df2['moistureOffPercent']
#df['Confidence'] = Confidence_weight*df1['Confidence'] + (1-Confidence_weight)*df2['Confidence']
df2.head();
df2.to_csv('submission.csv', index=False)

#df_issa3 = df.copy()

